In [4]:
import os
import pandas as pd

In [3]:
TITANIC_PATH = os.path.join("datasets", "titanic")

In [5]:
def load_titanic_data(filename, titanic_path=TITANIC_PATH):
    csv_path = os.path.join(titanic_path, filename)
    return pd.read_csv(csv_path)

In [6]:
test_data = load_titanic_data(r'test.csv')
train_data = load_titanic_data(r'train.csv')

In [8]:
test_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418 entries, 0 to 417
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  418 non-null    int64  
 1   Pclass       418 non-null    int64  
 2   Name         418 non-null    object 
 3   Sex          418 non-null    object 
 4   Age          332 non-null    float64
 5   SibSp        418 non-null    int64  
 6   Parch        418 non-null    int64  
 7   Ticket       418 non-null    object 
 8   Fare         417 non-null    float64
 9   Cabin        91 non-null     object 
 10  Embarked     418 non-null    object 
dtypes: float64(2), int64(4), object(5)
memory usage: 36.0+ KB


In [10]:
train_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


In [14]:
# Age, Cabin and Embarced are missing some values
# Name, Sex, Ticket, Cabin and Embarked are objects

In [29]:
import os

TITANIC_PATH = os.path.join('datasets', 'titanic')

In [30]:
import pandas as pd 

def load_titanic(filename, titanic_path = TITANIC_PATH):
    csv_path = os.path.join(titanic_path, filename)
    return pd.read_csv(csv_path)

In [31]:
train_data = load_titanic(r'train.csv')
test_data = load_titanic(r'test.csv') 

In [32]:
train_data = pd.DataFrame(train_data)
test_data = pd.DataFrame(test_data) 

In [33]:
test_data.head() 

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


In [34]:
train_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


In [35]:
train_data.describe()

,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare
count,891.000000,891.000000,891.000000,714.000000,891.000000,891.000000,891.000000
mean,446.000000,0.383838,2.308642,29.699118,0.523008,0.381594,32.204208
std,257.353842,0.486592,0.836071,14.526497,1.102743,0.806057,49.693429
min,1.000000,0.000000,1.000000,0.420000,0.000000,0.000000,0.000000
25%,223.500000,0.000000,2.000000,20.125000,0.000000,0.000000,7.910400
50%,446.000000,0.000000,3.000000,28.000000,0.000000,0.000000,14.454200
75%,668.500000,1.000000,3.000000,38.000000,1.000000,0.000000,31.000000
max,891.000000,1.000000,3.000000,80.000000,8.000000,6.000000,512.329200


In [36]:
train_data["Survived"].value_counts()

0    549
1    342
Name: Survived, dtype: int64

In [37]:
train_data["Embarked"].value_counts()

S    644
C    168
Q     77
Name: Embarked, dtype: int64

In [38]:
train_data["Sex"].value_counts()

male      577
female    314
Name: Sex, dtype: int64

In [61]:
X_train = train_data.drop("Survived", axis=1)
y_train = train_data["Survived"].copy()

In [40]:
X_train.shape

(891, 11)

In [41]:
from sklearn.base import BaseEstimator, TransformerMixin

class DataFrameSelector(BaseEstimator, TransformerMixin):
    def __init__(self, attribute_names):
        self.attribute_names = attribute_names
    def fit(self, X, y=None):
        return self
    def transform(self, X):
        return X[self.attribute_names]

In [42]:
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer

In [43]:
class MostFrequentImputer(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None):
        self.most_frequent_ = pd.Series([X[c].value_counts().index[0] for c in X],
                                        index=X.columns)
        return self
    def transform(self, X, y=None):
        return X.fillna(self.most_frequent_)

In [44]:
from sklearn.preprocessing import OneHotEncoder

In [45]:
cat_pipeline = Pipeline([
        ("frequent_imput", MostFrequentImputer()),
        ("cat_encoder", OneHotEncoder(sparse=False)),
    ])

In [46]:
num_selector = DataFrameSelector(["Age", "SibSp", "Parch", "Fare"])
cat_selector = DataFrameSelector(["Pclass", "Sex", "Embarked"])

In [47]:
from sklearn.compose import ColumnTransformer

num_attribs = num_selector.fit_transform(train_data)
cat_attribs = cat_selector.fit_transform(train_data)

preprocess_pipeline = ColumnTransformer([
        ("num", SimpleImputer(strategy="median"), ['Age', 'SibSp', 'Parch', 'Fare']),
        ("cat", cat_pipeline, ['Pclass', 'Sex', 'Embarked']),
    ])

In [73]:
X_train_processed = preprocess_pipeline.fit_transform(X_train)  
X_train_processed = pd.DataFrame(X_train_processed)
X_train_processed.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   0       891 non-null    float64
 1   1       891 non-null    float64
 2   2       891 non-null    float64
 3   3       891 non-null    float64
 4   4       891 non-null    float64
 5   5       891 non-null    float64
 6   6       891 non-null    float64
 7   7       891 non-null    float64
 8   8       891 non-null    float64
 9   9       891 non-null    float64
 10  10      891 non-null    float64
 11  11      891 non-null    float64
dtypes: float64(12)
memory usage: 83.7 KB


In [75]:
from sklearn.svm import SVC

svm_clf = SVC(gamma="auto") 
svm_clf.fit(X_train_processed, y_train)

SVC(gamma='auto')

In [76]:
X_test = preprocess_pipeline.transform(X_train)
y_pred = svm_clf.predict(X_test)

In [80]:
from sklearn.metrics import precision_score

precision_score(y_pred, y_train)

0.827485380116959

In [82]:
from sklearn.model_selection import cross_val_score

svm_scores = cross_val_score(svm_clf, X_train_processed, y_train, cv=10)
svm_scores.mean()

0.7329588014981274

In [87]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV

forest_clf = RandomForestClassifier()

param_grid = [{'n_estimators': [40, 80, 100, 120], 'bootstrap': [True]},
              {'n_estimators': [40, 80, 100, 120], 'bootstrap': [False]}]

grid_search = GridSearchCV(forest_clf, param_grid, cv=3, verbose=3)
grid_search.fit(X_train_processed, y_train)

Fitting 3 folds for each of 8 candidates, totalling 24 fits
[CV 1/3] END ................bootstrap=True, n_estimators=40; total time=   0.0s
[CV 2/3] END ................bootstrap=True, n_estimators=40; total time=   0.0s
[CV 3/3] END ................bootstrap=True, n_estimators=40; total time=   0.0s
[CV 1/3] END ................bootstrap=True, n_estimators=80; total time=   0.0s
[CV 2/3] END ................bootstrap=True, n_estimators=80; total time=   0.0s
[CV 3/3] END ................bootstrap=True, n_estimators=80; total time=   0.0s
[CV 1/3] END ...............bootstrap=True, n_estimators=100; total time=   0.0s
[CV 2/3] END ...............bootstrap=True, n_estimators=100; total time=   0.0s
[CV 3/3] END ...............bootstrap=True, n_estimators=100; total time=   0.0s
[CV 1/3] END ...............bootstrap=True, n_estimators=120; total time=   0.0s
[CV 2/3] END ...............bootstrap=True, n_estimators=120; total time=   0.0s
[CV 3/3] END ...............bootstrap=True, n_est

GridSearchCV(cv=3, estimator=RandomForestClassifier(),
             param_grid=[{'bootstrap': [True],
                          'n_estimators': [40, 80, 100, 120]},
                         {'bootstrap': [False],
                          'n_estimators': [40, 80, 100, 120]}],
             verbose=3)

In [88]:
grid_search.best_params_

{'bootstrap': True, 'n_estimators': 100}

In [93]:
forest_clf = RandomForestClassifier(n_estimators=100, random_state=42)

In [94]:
forest_scores = cross_val_score(forest_clf, X_train_processed, y_train, cv=10)
forest_scores.mean()

0.8126466916354558

In [106]:
forest_clf.fit(X_train_processed, y_train)

RandomForestClassifier(random_state=42)

In [110]:
test_data_processed = preprocess_pipeline.transform(test_data)

In [113]:
test_data_processed = pd.DataFrame(test_data_processed)
test_data_processed

,0,1,2,3,4,5,6,7,8,9,10,11
0,34.5,0.0,0.0,7.8292,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0
1,47.0,1.0,0.0,7.0000,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0
2,62.0,0.0,0.0,9.6875,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0
3,27.0,0.0,0.0,8.6625,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0
4,22.0,1.0,1.0,12.2875,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...
413,28.0,0.0,0.0,8.0500,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0
414,39.0,0.0,0.0,108.9000,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0
415,38.5,0.0,0.0,7.2500,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0
416,28.0,0.0,0.0,8.0500,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0


In [139]:
import numpy as np

In [148]:
predictions = forest_clf.predict(test_data_processed)
passengers = test_data["PassengerId"]

In [153]:
d = {'PassengerId': passengers, 'Survived': predictions}
submission_1 = pd.DataFrame(data=d)
submission_1

,PassengerId,Survived
0,892,0
1,893,0
2,894,0
3,895,1
4,896,1
...,...,...
413,1305,0
414,1306,1
415,1307,0
416,1308,0


In [152]:
submission_1.to_csv(r'C:\Users\Sasha\Desktop\Submission_titanic_1.csv', index=False)